# Lab 9: Spatial-Temporal patterns of Natural Disturbance
Building on the ForestFire model from previous lab, compute the fire iternval distribution (times between fire intervals) and the patch size distribution (size of forested patches in steady-state landscape pattern)


In [ ]:
import numpy as np

### Create Continuous Patches
This is a surprisingly challenging problem to solve in the general case given how good our visual system is at identifying them!
The idea I had here was to start by giving each occupied cell a unique value, then "grow" patches from occupied cells by allowing the smallest of these unique values to propogate to neighbouring cells.  Repeat until the propogation is finished.


In [ ]:
neighbourhood = np.array([
    [0, 1, 0],
    [1, 1, 1],
    [0, 1, 0],
])

def min_neighbour(a):
    """ Return the smallest non-zero neighbourhood value or 0 if centre cell is a zero """
    p = a*neighbourhood
    centre = tuple(d//2 for d in a.shape)
    return np.min(p[p>0]) if a[centre] else 0

def consolidate(array):
    """ return copy of array with adjacent cells consolidated into a patch with the lowest value among occupied neighbours """
    rows, cols = array.shape
    k = neighbourhood.shape[0]
    array = np.pad(array, 1, 'constant')
    return np.array([
        [min_neighbour(array[row:row+k, col:col+k]) for col in range(cols) ]
            for row in range(rows)
    ])

def patchify(array, category):
    """ Return an array with each contiguous patch identified by a unique integer
    array:  array of int categorical values
    category: the int category value to identify patches

    return: array of same shape with a unique value identifying cells in each patch and zeros elsewhere
    """
    patches = np.zeros(array.shape, dtype=np.uint)
    patches[array==category] = range(1, len(array[array==category])+1)

    patches_growing = np.array([True,])
    while np.any(patches_growing):
        prev_patches = patches
        patches = consolidate(prev_patches)
        patches_growing = patches != prev_patches  # patches are growning until consolidate algorithm stablaizes.

    return patches